# Importing Libraries

In [1]:
# loading libraries
# keras
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.wrappers.scikit_learn import KerasClassifier, KerasRegressor
from keras import regularizers
from keras.datasets import fashion_mnist
from keras.layers.normalization import BatchNormalization

# sklearn
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedShuffleSplit

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


Using TensorFlow backend.


# Task 1

In [3]:
# loading the data from sklearn digits
digits = load_digits()
print(digits.data.shape)

X = digits.data
y = digits.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, stratify = y, shuffle = True)

num_classes = 10
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

(1797, 64)


In [0]:
# creating an initial model with 2 hidden layers and regularization paratmer = 0.01
def create_model(reg_parameter = 0.01, hidden_size = 32):  
  model = Sequential([
      Dense(hidden_size, input_shape=(64,), kernel_regularizer = regularizers.l2(reg_parameter)),
      Activation('relu'),
      Dense(int(hidden_size/2), kernel_regularizer = regularizers.l2(reg_parameter)),
      Activation('relu'),
      Dense(10),
      Activation('softmax')])

  model.compile("adam", "categorical_crossentropy", metrics=['accuracy'])
  return model

In [5]:
model = create_model()
model.fit(X_train, y_train, batch_size=128, epochs=10, verbose=1)

Epoch 1/10
1437/1437 [==============================] - 2s 1ms/step - loss: 4.3591 - accuracy: 0.2345
Epoch 2/10
1437/1437 [==============================] - 0s 28us/step - loss: 2.8547 - accuracy: 0.3166
Epoch 3/10
1437/1437 [==============================] - 0s 26us/step - loss: 2.3468 - accuracy: 0.3904
Epoch 4/10
1437/1437 [==============================] - 0s 23us/step - loss: 2.0603 - accuracy: 0.4788
Epoch 5/10
1437/1437 [==============================] - 0s 22us/step - loss: 1.8496 - accuracy: 0.5623
Epoch 6/10
1437/1437 [==============================] - 0s 26us/step - loss: 1.6686 - accuracy: 0.6472
Epoch 7/10
1437/1437 [==============================] - 0s 25us/step - loss: 1.5164 - accuracy: 0.7015
Epoch 8/10
1437/1437 [==============================] - 0s 23us/step - loss: 1.3778 - accuracy: 0.7474
Epoch 9/10
1437/1437 [==============================] - 0s 22us/step - loss: 1.2442 - accuracy: 0.7898
Epoch 10/10
1437/1437 [==============================] - 0s 22us/step - lo

In [6]:
score = model.evaluate(X_test, y_test, verbose=0)
print("Test loss: {:.3f}".format(score[0]))
print("Test Accuracy: {:.3f}".format(score[1]))

Test loss: 1.024
Test Accuracy: 0.850


In [7]:
# grid search for l2 regularization parameter and hidden layer size
# create model
model = KerasClassifier(build_fn=create_model, epochs=10, batch_size=128, verbose=0)
# define the grid search parameters
reg_param_list = [0.001, 0.01, 0.05, 0.1, 0.5, 1, 10, 100]
hidden_layers = [16, 32, 64, 128, 256]
param_grid = dict(hidden_size=hidden_layers, reg_parameter=reg_param_list)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv = StratifiedShuffleSplit(n_splits = 1, test_size = 0.2))
grid_result = grid.fit(X_train, y_train)
# summarize results
print("Best training score: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
print("Testing score: %f" % grid.score(X_test, y_test))

Best training score: 0.972222 using {'hidden_size': 256, 'reg_parameter': 0.001}
Testing score: 0.988889


In [8]:
res = pd.DataFrame(grid.cv_results_)
res.pivot_table(index=["param_reg_parameter", "param_hidden_size"],
                values=["mean_test_score"])

mean_test_score
param_reg_parameter param_hidden_size                 
0.001               16                        0.413194
                    32                        0.635417
                    64                        0.923611
                    128                       0.916667
                    256                       0.972222
0.010               16                        0.281250
                    32                        0.486111
                    64                        0.892361
                    128                       0.954861
                    256                       0.965278
0.050               16                        0.170139
                    32                        0.454861
                    64                        0.875000
                    128                       0.934028
                    256                       0.958333
0.100               16                        0.284722
                    32                        0.555556
                    64                        0.798611
                    128                       0.947917
                    256                       0.937500
0.500               16                        0.250000
                    32                        0.527778
                    64                        0.815972
                    128                       0.940972
                    256                       0.923611
1.000               16                        0.222222
                    32                        0.291667
                    64                        0.704861
                    128                       0.878472
                    256                       0.892361
10.000              16                        0.201389
                    32                        0.166667
                    64                        0.159722
                    128                       0.444444
                    256                       0.722222
100.000             16                        0.131944
                    32                        0.142361
                    64                        0.281250
                    128                       0.333333
                    256                       0.534722